# Outside FinSpace Cluster Management
Clusters can be managed form outside of FinSpace as well as this notebook will demonstrate.

In [ ]:
import datetime as dt
import pandas as pd 
import boto3
import time
import pprint

from botocore.config import Config

In [ ]:
#### ---------------------------------------------------
#### Identify your region
#### ---------------------------------------------------
region_name  = 'us-east-1'

### ----------------------------------------------------------------
### Get Credentials from the "API Credentials" in FinSpace 
### ----------------------------------------------------------------
hab_access_key_id     = ''
hab_secret_access_key = ''
hab_session_token     = ''


# Python Helper
This class wraps the service API so its easier to consume those results.

In [ ]:
%load ../Utilities/finspace.py

## Initialize the Python Helper Class


In [ ]:
hab_session = boto3.session.Session(
    region_name           = region_name,
    aws_access_key_id     = hab_access_key_id,
    aws_secret_access_key = hab_secret_access_key,
    aws_session_token     = hab_session_token
)

finspace = FinSpace(
    boto_session = hab_session,
    dev_overrides = {
        "region_name" : region_name 
    }
)

# Cluster Management

functions available fro the fiunspace client:
- list_clusters
- create_cluster
- update_cluster
- terminate_cluster
- get_cluster_connection_details

In [ ]:
t = finspace.list_clusters()
pd.DataFrame.from_dict(t)

In [ ]:
# locate the running cluster and resize it
active_clusters = finspace.list_clusters( ['RUNNING', 'UPDATING'] )
pd.DataFrame.from_dict(active_clusters)

In [ ]:
if len(active_clusters) > 0:
    cid = active_clusters[0]['clusterId']

    this_cluster = finspace.get_cluster( clusterId=cid )
    print( pd.DataFrame.from_dict(this_cluster) )
else:
    print("no clusters running")

In [ ]:
finspace.wait_for_status( clusterId = cid, status = 'RUNNING' )

In [ ]:
# to change a cluster's size, call update
finspace.update_cluster(clusterId = cid, template = 'Medium')

In [ ]:
finspace.wait_for_status( clusterId = cid, status = 'RUNNING' )